### LSTM 을 활용한 주가 예측 모델
##### 이동평균선만 추가

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import datetime as dt


%matplotlib inline
warnings.filterwarnings('ignore')

sqlite DB에서 데이터 불러오기

In [2]:
import sqlite3
con = sqlite3.connect(r"C:\Users\구남이\OneDrive\바탕 화면\활동들\한이음 멘토링\stock.db")
df = pd.read_sql("SELECT * FROM '039490'", con, index_col=None)
df.rename(columns={'index':'date'}, inplace=True)
df = df.set_index('date')
df.index = pd.DatetimeIndex(df.index)
df.sort_index(ascending=True, inplace=True)
df

,open,high,low,close,volume
date,,,,,
2004-04-23,6076,6799,5353,5353,1415109
2004-04-26,5436,5741,5261,5398,343283
2004-04-27,5452,5482,5033,5063,299489
2004-04-28,5071,5452,4995,5444,182604
2004-04-29,5117,5436,5025,5101,109911
...,...,...,...,...,...
2022-05-11,87500,87500,85500,85800,108071
2022-05-12,84700,85300,82700,83200,104007
2022-05-13,84000,87400,83500,85800,86525


In [3]:
# 이동평균선 추가
ma = [5,20,60,120]
for days in ma:
    df['ma_'+str(days)] = df['close'].rolling(window = days).mean()

df.dropna(inplace=True)
df 

,open,high,low,close,volume,ma_5,ma_20,ma_60,ma_120
date,,,,,,,,,
2004-10-14,3959,4066,3906,4035,171655,3940.8,3963.60,3421.083333,3346.366667
2004-10-15,4066,4096,3959,3990,128530,3959.2,3946.85,3442.283333,3335.008333
2004-10-18,3997,3997,3860,3868,77233,3944.0,3935.05,3461.383333,3322.258333
2004-10-19,3868,3914,3830,3853,63272,3935.0,3922.90,3481.183333,3312.175000
2004-10-20,3807,3853,3769,3769,118227,3903.0,3909.60,3499.450000,3298.216667
...,...,...,...,...,...,...,...,...,...
2022-05-11,87500,87500,85500,85800,108071,89460.0,94620.00,97348.333333,100015.833333
2022-05-12,84700,85300,82700,83200,104007,87480.0,93785.00,97120.000000,99788.333333
2022-05-13,84000,87400,83500,85800,86525,86360.0,93050.00,96945.000000,99603.333333


In [4]:
df.drop('close', axis=1)

,open,high,low,volume,ma_5,ma_20,ma_60,ma_120
date,,,,,,,,
2004-10-14,3959,4066,3906,171655,3940.8,3963.60,3421.083333,3346.366667
2004-10-15,4066,4096,3959,128530,3959.2,3946.85,3442.283333,3335.008333
2004-10-18,3997,3997,3860,77233,3944.0,3935.05,3461.383333,3322.258333
2004-10-19,3868,3914,3830,63272,3935.0,3922.90,3481.183333,3312.175000
2004-10-20,3807,3853,3769,118227,3903.0,3909.60,3499.450000,3298.216667
...,...,...,...,...,...,...,...,...
2022-05-11,87500,87500,85500,108071,89460.0,94620.00,97348.333333,100015.833333
2022-05-12,84700,85300,82700,104007,87480.0,93785.00,97120.000000,99788.333333
2022-05-13,84000,87400,83500,86525,86360.0,93050.00,96945.000000,99603.333333


In [5]:
df['close']

date
2004-10-14     4035
2004-10-15     3990
2004-10-18     3868
2004-10-19     3853
2004-10-20     3769
              ...  
2022-05-11    85800
2022-05-12    83200
2022-05-13    85800
2022-05-16    85900
2022-05-17    87600
Name: close, Length: 4352, dtype: int64

In [6]:
from sklearn.preprocessing import MinMaxScaler
# 피처값 스케일링
scaler = MinMaxScaler()

scaled_df = scaler.fit_transform(df.drop('close', axis=1))
scaled_df = pd.DataFrame(scaled_df, columns = df.drop('close', axis=1).columns)

In [7]:
# 타켓 스케일링
scaler1 = MinMaxScaler()
scaled_df['close'] = scaler1.fit_transform(df['close'].values.reshape(-1,1))

In [8]:
scaled_df

,open,high,low,volume,ma_5,ma_20,ma_60,ma_120,close
0,0.003777,0.004016,0.003642,0.119625,0.003413,0.003428,0.000000,0.000860,0.004173
1,0.004449,0.004199,0.003990,0.089071,0.003534,0.003313,0.000154,0.000774,0.003890
2,0.004015,0.003596,0.003341,0.052727,0.003434,0.003232,0.000294,0.000677,0.003121
3,0.003205,0.003090,0.003144,0.042836,0.003374,0.003148,0.000438,0.000601,0.003026
4,0.002821,0.002718,0.002745,0.081771,0.003163,0.003057,0.000571,0.000495,0.002496
...,...,...,...,...,...,...,...,...,...
4347,0.528723,0.512472,0.538159,0.074576,0.567238,0.626130,0.684249,0.734314,0.519628
4348,0.511128,0.499065,0.519817,0.071697,0.554184,0.620395,0.682586,0.732588,0.503237
4349,0.506730,0.511862,0.525057,0.059311,0.546800,0.615346,0.681311,0.731184,0.519628
4350,0.524953,0.513081,0.539469,0.037247,0.542712,0.611156,0.679647,0.729913,0.520258


In [9]:
#train,test 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_df.drop('close', 1), scaled_df['close'], test_size=0.3, random_state=0, shuffle=False)

In [10]:
# tensorflow dataset을 활용한 시퀀스 데이터셋 구성
import tensorflow as tf
def windowed_dataset(series, window_size, batch_size, shuffle):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

In [11]:
# hyper parameter 정의 
WINDOW_SIZE=20
BATCH_SIZE=32

In [12]:
# trian_data는 학습용 데이터셋, test_data는 검증용 데이터셋
train_data = windowed_dataset(y_train, WINDOW_SIZE, BATCH_SIZE, True)
test_data = windowed_dataset(y_test, WINDOW_SIZE, BATCH_SIZE, False)


In [13]:
 # 모델 생성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential([
    # 1차원 feature map 생성
    Conv1D(filters=32, kernel_size=5,
        padding="causal",
        #활성화함수를 relu로 정의
        activation="relu",
        input_shape=[WINDOW_SIZE, 1]),
    # LSTM
    LSTM(16, activation='tanh'),    
    Dense(16, activation="relu"),
    Dense(1)])

loss = Huber()
optimizer = Adam(0.005) ##3compile 시 optimizer는 adam으로 사용
model.compile(loss=Huber(), optimizer=optimizer, metrics=['mse'])

earlystopping = EarlyStopping(monitor='val_loss', patience=10)

filename = os.path.join('tmp', 'ckeckpointer.ckpt')
checkpoint = ModelCheckpoint(filename, 
                            save_weights_only=True,
                            save_best_only=True, 
                            monitor='val_loss',
                            verbose=1)

In [14]:
# 모델 학습
history = model.fit(train_data, 
                validation_data=(test_data), 
                epochs=50,   #50번 학습
                callbacks=[checkpoint, earlystopping])

Epoch 1/50
95/95 [==============================] - 15s 64ms/step - loss: 8.7821e-04 - mse: 0.0018 - val_loss: 7.7610e-04 - val_mse: 0.0016

Epoch 00001: val_loss improved from inf to 0.00078, saving model to tmp\ckeckpointer.ckpt
Epoch 2/50
95/95 [==============================] - 4s 42ms/step - loss: 1.5881e-04 - mse: 3.1763e-04 - val_loss: 6.2303e-04 - val_mse: 0.0012

Epoch 00002: val_loss improved from 0.00078 to 0.00062, saving model to tmp\ckeckpointer.ckpt
Epoch 3/50
95/95 [==============================] - 4s 37ms/step - loss: 1.9337e-04 - mse: 3.8674e-04 - val_loss: 0.0011 - val_mse: 0.0022

Epoch 00003: val_loss did not improve from 0.00062
Epoch 4/50
95/95 [==============================] - 4s 38ms/step - loss: 1.2741e-04 - mse: 2.5482e-04 - val_loss: 0.0013 - val_mse: 0.0025

Epoch 00004: val_loss did not improve from 0.00062
Epoch 5/50
95/95 [==============================] - 4s 35ms/step - loss: 1.3223e-04 - mse: 2.6447e-04 - val_loss: 0.0015 - val_mse: 0.0029

Epoch 000

In [15]:
pred = model.predict(test_data)

In [16]:
rescaled_y_test = scaler1.inverse_transform(np.array(y_test).reshape(-1, 1))
rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))

In [17]:
# 평가지표 함수
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

def confirm_result(y_test, y_pred):
    MAE = mean_absolute_error(y_test, y_pred)
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    MSLE = mean_squared_log_error(y_test, y_pred)
    RMSLE = np.sqrt(mean_squared_log_error(y_test, y_pred))
    R2 = r2_score(y_test, y_pred)
        
    pd.options.display.float_format = '{:.5f}'.format
    Result = pd.DataFrame(data=[MAE,RMSE, RMSLE, R2],
                            index = ['MAE','RMSE', 'RMSLE', 'R2'],
                            columns=['Results'])
    return Result

In [18]:
confirm_result(rescaled_y_test[20:], rescaled_pred)

,Results
MAE,5094.21152
RMSE,6735.85523
RMSLE,0.06122
R2,0.89119
